# Web Scrapping Assignment 3

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from datetime import datetime

### Q.NO.1 Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [42]:
driver=webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
driver.get('http://amazon.in/')
driver.implicitly_wait(5)

#enter the details in the search column
product_vertical=input("Enter product vertical that needs to be searched:")
driver.find_element_by_id("twotabsearchtextbox").send_keys(product_vertical)
driver.find_element_by_id("nav-search-submit-button").click()

Enter product vertical that needs to be searched:guitars


### Q.NO.2 In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [43]:
#create a function to get the details of all products of first three pages
urls=[]
for k in range(3):
    for i in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-medium']//h2"):
        urls.append(i.find_element_by_xpath(".//a").get_attribute('href'))
    try:
        next_page=driver.find_element_by_xpath("//div[@class='a-text-center']/ul/li[@class='a-last']/a").get_attribute('href')
        driver.get(next_page)
        driver.refresh()   
    except:
        break
        
        
#create empty list to store data
brand_name=[]
name_of_the_product=[]
rating=[]
no_of_ratings=[]
price=[]
expected_delivery=[]
return_exchange=[]
other_details=[]
availability=[]
product_url=[]

#traverse all URL one by one
for i in urls:
    driver.get(i)
    product_url.append(i)
    driver.implicitly_wait(5)
    
    
#get the brand name
    try: brand_name.append(driver.find_element_by_id("productTitle").text.split(' ')[0]) 
    except: brand_name.append('-')
    driver.implicitly_wait(5)
    
#get the name of product
    try: 
        string=''
        for i in driver.find_element_by_id("productTitle").text.split(' ')[1:]:
            string=string+' '+i
        name_of_the_product.append(string)
    except: name_of_the_product.append('-')
    driver.implicitly_wait(5)
    
#scrape the rating of the product
    try: rating.append(driver.find_element_by_id("acrPopover").get_attribute('title')) 
    except: rating.append('-')
    driver.implicitly_wait(5)
    
#scrape the number of ratings
    try: no_of_ratings.append(driver.find_element_by_id("acrCustomerReviewText").text) 
    except: no_of_ratings.append('-')
    driver.implicitly_wait(5)
    
#scrape the price of the product
    try: 
        try: price.append(driver.find_element_by_id("priceblock_saleprice").text) 
        except:
            try: price.append(driver.find_element_by_id("priceblock_dealprice").text)
            except: price.append(driver.find_element_by_id("priceblock_ourprice").text)
    except: price.append('-')
    driver.implicitly_wait(5)
    
#scrape the details of return/exchnage of the product
    try: 
        return_row=driver.find_element_by_xpath("//div[@class='a-row icon-farm-wrapper']").find_elements_by_xpath(".//div")
        found=False
        for i in return_row:
            if(i.get_attribute('data-name')=='RETURNS_POLICY'):
                found=True
                return_exchange.append(i.find_element_by_xpath(".//span[1]/div[2]/a").text)
        if(found==False):
            return_exchange.append('-')
    except: return_exchange.append('-')
    driver.implicitly_wait(5)
    
#scrape the expected delivery date
    try: expected_delivery.append(driver.find_element_by_id("ddmDeliveryMessage").find_element_by_xpath(".//b").text)
    except: expected_delivery.append('-')
    driver.implicitly_wait(5)
    
#scrape the availablity of the product
    try: 
        try: availability.append(driver.find_element_by_id("availability").find_element_by_xpath(".//span").text)
        except: availability.append(driver.find_element_by_id("deal_availability").find_element_by_xpath(".//div/span").text)
    except: availability.append('-')
    driver.implicitly_wait(5)
    
#scrape other details
    try:
        details=[i.text.replace('\n','---') for i in driver.find_element_by_id("productDetails_techSpec_section_1").find_elements_by_xpath(".//tbody")] 
        other_details.append(details[0])
    except: other_details.append('-')
    driver.implicitly_wait(5)

#create a DataFrame for our data
amazon=pd.DataFrame({'Brand Name':brand_name,'Name of Product':name_of_the_product,'Rating':rating,'No. of Ratings':no_of_ratings,'Price':price,
             'Return/Exchange':return_exchange,'Expected Delivery':expected_delivery,'Availability':availability,'Other details':other_details,
             'Product URL':product_url})

#save the dataframe in csv file
amazon.to_csv('products.csv')

In [44]:
amazon

,Brand Name,Name of Product,Rating,No. of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Other details,Product URL
0,Kadence,Slowhand Premium Jumbo Semi Acoustic Guitar w...,4.1 out of 5 stars,302 ratings,"₹10,499.00",7 Days Replacement,"Monday, Dec 20",In stock.,Brand Kadence---Manufacturer Kadence---Model S...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence,"Frontier Series, Acoustic Guitar With/Wihtout...",4.0 out of 5 stars,"1,238 ratings","₹5,199.00",7 Days Replacement,"Monday, Dec 20",In stock.,Brand Kadence---Manufacturer Kadence---Model F...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Juarez,"Acoustic Guitar, 38 Inch Curved Body Cutaway,...",3.9 out of 5 stars,"1,296 ratings","₹2,190.00",7 Days Replacement,"Friday, Dec 17",In stock.,Product Dimensions 89 x 43.6 x 11.4 cm; 2.1 Ki...,https://www.amazon.in/gp/slredirect/picassoRed...
3,Vault,EA40 Premium 41 inch Spruce-Top Cutaway Acous...,3.9 out of 5 stars,234 ratings,"₹6,990.00",7 Days Replacement,"Saturday, Dec 18",In stock.,Brand VAULT---Manufacturer VAULT---Model EA40N...,https://www.amazon.in/gp/slredirect/picassoRed...
4,Juârez,"Acoustic Guitar, 38 Inch Cutaway, 038C with B...",4.0 out of 5 stars,"14,254 ratings","₹2,649.00",7 Days Replacement,"Saturday, Dec 18",In stock.,Product Dimensions 98 x 40 x 10 cm; 2.06 Kilog...,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
178,"Granada,","Acoustic Guitar, Cutaway With EQ, PRG-118CEQ ...",4.8 out of 5 stars,5 ratings,"₹9,891.00",7 Days Replacement,"Wednesday, Dec 22",In stock.,Product Dimensions 104 x 48 x 13.7 cm; 2.16 Ki...,https://www.amazon.in/Granada-Acoustic-Cutaway...
179,Festra,"38C, 38 Inch Acoustic Guitar Kit with Bag, St...",-,-,"₹2,695.00",7 Days Replacement,"Sunday, Dec 19",In stock.,Brand Festra---Manufacturer FOSHAN SANSHUI JUN...,https://www.amazon.in/gp/slredirect/picassoRed...
180,Blueberry,"B-40, 40"" Acoustic Guitar Kit with Bag,Belt, ...",1.8 out of 5 stars,3 ratings,"₹3,595.00",7 Days Replacement,"Sunday, Dec 19",In stock.,Product Dimensions 94 x 34 x 9 cm; 2 Kilograms...,https://www.amazon.in/gp/slredirect/picassoRed...
181,Festra,"38C, 38 Inch Acoustic Guitar Kit with Bag, St...",-,-,"₹2,695.00",7 Days Replacement,"Sunday, Dec 19",In stock.,Brand Festra---Manufacturer FOSHAN SANSHUI JUN...,https://www.amazon.in/gp/slredirect/picassoRed...


### Q.NO.3 Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [45]:
#get the web page with given url
driver = webdriver.Chrome("chromedriver.exe")
url = "https://images.google.com/"
urls = []    #image urls to download
data = []
search_item = ["fruits", "cars", "Machine Learning"]
for item in search_item:
    driver.get(url)  #google search for items
    time.sleep(5)
    search_bar = driver.find_element_by_tag_name("input") #finding search bar
    
    search_bar.send_keys(str(item))      #send search item
    
    search_button =driver.find_element_by_xpath("//button[@class='Tg7LZd']").click() #lets click on search button
    
    # scroll the page in order to generate more images on the website
    for _ in range(500):
        driver.execute_script("window.scrollBy(0,100)")
        
        imgs = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:100]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 300:
        break
    print("Downloading {0} of {1} images" .format(i, 300))
    response= requests.get(urls[i])

    file = open(r"C:\Users\my smile\Desktop\Google images\images"+str(i)+".jpg", "wb")

    file.write(response.content)

### Q. NO. 4 Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [46]:
driver = webdriver.Chrome("chromedriver.exe")
#get the web page with given url
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(3)

cancle_btn = driver.find_element_by_xpath("//div[@class='_2QfC02']//button").click()
#giving inputs to search bar
serch_bar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']//input")
serch_bar.send_keys("smartphone")

srch_btn = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
srch_btn.click()
time.sleep(5)

In [47]:
#fetching urls of every smartphone
urls = []
for i in driver.find_elements_by_xpath("//a[@class='_1fQZEK']"):
    urls.append(i.get_attribute("href"))

#fetching required data
brand_names = []
name = []
color = []
RAM = []
storage = []
prim_cam = []
sec_cam = []
Display_size = []
display_resolution = []
processor = []
processor_core = []
battery_cap = []
prices = []
product_url = []
#fetching Battery capacity
for i in driver.find_elements_by_xpath("//div[@class='fMghEO']"):
    try:
        bat_cap = driver.find_element_by_xpath("//ul[@class='_1xgFaf']//li[4]")
        battery_cap.append(bat_cap.text)
    except NoSuchElementException:
        battery_cap.append('-')
for i in urls:
    driver.get(i)
    time.sleep(3)
    
    #fetching brand names
    try:
        br_name = driver.find_element_by_xpath("/html/body/div/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/div[4]/a")
        brand_names.append(br_name.text.replace('Mobiles',''))
    except NoSuchElementException:
        brand_names.append('-')
    #fetching smartphone names
    try:
        smt_name = driver.find_element_by_xpath("//h1[@class='yhB1nd']//span")
        name.append(smt_name.text)
    except NoSuchElementException:
        name.append('-')
    
    #fetching colors of smartphone
    try:
        clr = driver.find_element_by_xpath("//table[@class='_14cfVK']//tr[4]//td[2]")
        color.append(clr.text)
    except NoSuchElementException:
        color.append('-')
    time.sleep(2)
    #readmore button
    try:
        read_more_btn = driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
    except NoSuchElementException:
        pass
    time.sleep(3)
    
    #fetching Display size
    try:
        disp_size = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[2]//tr[1]//td[2]")
        Display_size.append(disp_size.text)
    except NoSuchElementException:
        Display_size.append
        
    #fetching display_resolution
    try:
        disp_res = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[2]//tr[2]//td[2]")
        display_resolution.append(disp_res.text)
    except NoSuchElementException:
        display_resolution.append('-')
        
    #fetching processor info
    try:
        pro = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[3]//tr[2]//td[2]")
        processor.append(pro.text)
    except NoSuchElementException:
        processor.append('-')
    #ferching processor core info
    try:
        pro_core = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[3]//tr[3]//td[2]")
        processor_core.append(pro_core.text)
    except NoSuchElementException:
        processor_core.append('-')
    #fetching storage/ROM
    try:
        rom = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[4]//tr[1]//td[2]")
        storage.append(rom.text)
    except NoSuchElementException:
        storage.append('-')
    #fetching RAM info
    try:
        ram = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[4]//tr[2]//td[2]")
        RAM.append(ram.text)
    except NoSuchElementException:
        RAM.append('-')
    time.sleep(2)
    
    #fetching primary camera info
    try:
        p_cam = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[5]//tr[2]//td[2]")
        prim_cam.append(p_cam.text)
    except NoSuchElementException:
        prim_cam.append('-')
    #fetching secondary camera info
    try:
        s_cam = driver.find_element_by_xpath("//div[@class='_1UhVsV']//div[5]//tr[5]//td[2]")
        sec_cam.append(s_cam.text)
    except NoSuchElementException:
        sec_cam.append('-')
    
    
   
    #fetching price of smartphone
    try:
        price = driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")
        prices.append(price.text.replace('₹','Rs. '))
    except NoSuchElementException:
        prices.append('-')
    
    
#creating dataframe
data = list(zip(brand_names,name,color,RAM,storage,prim_cam,sec_cam,Display_size,display_resolution,processor,processor_core,battery_cap,prices,urls))
df = pd.DataFrame(data, columns = ["Brand Name","Smartphone Name","color","RAM","Storage/ROM","primary Camera","Secondary Camera","Display size","Display Resolution","Processor","Processor-core","Battery Capacity","Price","Product url"])

#passing dataframe to csv file
df.to_csv(r"C:\Users\my smile\Desktop\flipkart_smartphones.csv")
#print dataframe
df

,Brand Name,Smartphone Name,color,RAM,Storage/ROM,primary Camera,Secondary Camera,Display size,Display Resolution,Processor,Processor-core,Battery Capacity,Price,Product url
0,-,"realme C21Y (Cross Blue, 64 GB) (4 GB RAM)",Cross Blue,4 GB,64 GB,13MP + 2MP + 2MP,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Unisoc T610,Octa Core,5000 mAh Battery,"Rs. 9,999",https://www.flipkart.com/realme-c21y-cross-blu...
1,-,"realme C21Y (Cross Blue, 32 GB) (3 GB RAM)",Cross Blue,3 GB,32 GB,13MP + 2MP + 2MP,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Unisoc T610,Octa Core,5000 mAh Battery,"Rs. 8,999",https://www.flipkart.com/realme-c21y-cross-blu...
2,-,"realme C21Y (Cross Black, 64 GB) (4 GB RAM)",Cross Black,4 GB,64 GB,13MP + 2MP + 2MP,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Unisoc T610,Octa Core,5000 mAh Battery,"Rs. 9,999",https://www.flipkart.com/realme-c21y-cross-bla...
3,-,"GIONEE Max Pro (Black, 32 GB) (3 GB RAM)",Black,3 GB,32 GB,13MP + 2MP,8MP Front Camera,16.56 cm (6.52 inch),1560 x 720 Pixels,Octa Core,Octa Core,5000 mAh Battery,"Rs. 7,299",https://www.flipkart.com/gionee-max-pro-black-...
4,-,"realme C25Y (Glacier Blue, 64 GB) (4 GB RAM)",Glacier Blue,4 GB,64 GB,50MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Unisoc T610 Octa Core,Octa Core,5000 mAh Battery,"Rs. 10,999",https://www.flipkart.com/realme-c25y-glacier-b...
5,-,"realme C25Y (Metal Grey, 64 GB) (4 GB RAM)",Metal Grey,4 GB,64 GB,50MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,Unisoc T610 Octa Core,Octa Core,5000 mAh Battery,"Rs. 10,999",https://www.flipkart.com/realme-c25y-metal-gre...
6,-,"realme Narzo 50i (Mint Green, 32 GB) (2 GB RAM)",Mint Green,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,SC9863A,Octa Core,5000 mAh Battery,"Rs. 7,499",https://www.flipkart.com/realme-narzo-50i-mint...
7,-,"realme Narzo 50i (Carbon Black, 32 GB) (2 GB ...",Carbon Black,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,SC9863A,Octa Core,5000 mAh Battery,"Rs. 7,499",https://www.flipkart.com/realme-narzo-50i-carb...
8,-,"REDMI 9i (Nature Green, 64 GB) (4 GB RAM)",Nature Green,4 GB,64 GB,13MP Rear Camera,5MP Front Camera,16.59 cm (6.53 inch),1600 x 720 Pixels,MediaTek Helio G25,Octa Core,5000 mAh Battery,"Rs. 8,799",https://www.flipkart.com/redmi-9i-nature-green...
9,-,"realme Narzo 50i (Mint Green, 64 GB) (4 GB RAM)",Mint Green,4 GB,64 GB,8MP Rear Camera,5MP Front Camera,16.51 cm (6.5 inch),1600 x 720 Pixels,SC9863A,Octa Core,5000 mAh Battery,"Rs. 8,999",https://www.flipkart.com/realme-narzo-50i-mint...


### Q. NO.5 Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [48]:
driver=webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
driver.get('https://www.google.com/maps/')
driver.implicitly_wait(5)

#create a function to enter the city name 
city = input('Enter City Name : ') 
driver.find_element_by_id('searchboxinput').send_keys(city)
    
#create a fucntion to search the result
driver.find_element_by_id('searchboxinput').send_keys(Keys.ENTER)
driver.implicitly_wait(6)

#Scrape the entered city latitude and Longitude from google map
lat=driver.current_url.split('/@')[1].split(',')[0]
long=driver.current_url.split('/@')[1].split(',')[1].split(',')[0]
print('Latitude:',lat,'Longitude:',long)

Enter City Name : Hyderabad
Latitude: 22.4168315 Longitude: 71.319437


### Q.NO. 6 Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 –September 20) from trak.in

In [49]:
#load the drivers and URL
driver=webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
driver.get('https://trak.in/')
driver.implicitly_wait(5)

#enter the details in the search column
search_button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(search_button)

#create an empty list to store data
Date=[]
Startup=[]
Industry=[]
SubVertical=[]
Location=[]
Investor=[]
Investment=[]
Amount=[]

In [50]:
#define a function to get the details
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()
    
#scrape the date
    date = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in date:
        Date.append(d.text)
        
#scrape the detaisl
    startup = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in startup:
        Startup.append(n.text)
    
#scrape the industry details
    industry = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in industry:
        Industry.append(n.text)
    
#scrape the subvertical details
    subvertical = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in subvertical:
        SubVertical.append(s.text)

#scrape the location details
    location = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in location:
        Location.append(l.text)
    
#scrape the investors detils
    investor = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in investor:
        Investor.append(n.text)
    
#scrape the investment details
    investment = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in investment:
        Investment.append(n.text)
    
#scrape the amount
    amtount = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amtount:
        Amount.append(a.text)

#create a DataFrame for our data
trak_fund=pd.DataFrame({})
trak_fund['Date']=Date
trak_fund['Startup']=Startup 
trak_fund['Industry']=Industry
trak_fund['SubVertical']=SubVertical
trak_fund['Location']=Location
trak_fund['Investor']=Investor
trak_fund['Investment']=Investment
trak_fund['Amount']=Amount

trak_fund

,Date,Startup,Industry,SubVertical,Location,Investor,Investment,Amount
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


### Q.NO. 7 Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [51]:
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException #importing exception
driver=webdriver.Chrome(r'chromedriver.exe')
driver.get('https://www.digit.in')
search_laptop_button = driver.find_element_by_xpath('//div[@class="search"]')
#finding the search button and clicking
search_laptop_button.click()
search_bar=driver.find_element_by_id("globalPageSearchText")     
search_bar.send_keys('top 10 gaming laptop')
search_bar.send_keys(Keys.ENTER)
#finding the search button and clicking
top10=driver.find_element_by_id("content_top10") 
top10.click()
#finding the search button and clicking
best_laptop=driver.find_element_by_xpath("//*[text()='Best Gaming Laptops in India']") 
best_laptop.click()

In [52]:
#creating empty list
Product=[]
Processor=[]
Memory=[]
OS=[]
Display=[]
Seller=[]
Price=[]
Availability=[]


In [53]:
#scraping all the details
Brand_tag=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in Brand_tag:
    try:
        brandtag=i.text
        Product.append(brandtag)
    except NoSuchElementException   as e:
        Product.append('-')
        
specification = driver.find_elements_by_xpath('//div[@class="Specs-Wrap"]/ul')
for details in specification:
    lap_spec= details.find_elements_by_xpath('.//div[@class="value"]')
    try:
        OS.append(lap_spec[0].text)
        Display.append(lap_spec[1].text)
        Processor.append(lap_spec[2].text)
        Memory.append(lap_spec[3].text)
    except NoSuchElementException   as e:
        OS.append('-')
        Displah.append('-')
        Processor.append('-')
        Memory.append('-')
        
seller_info= driver.find_elements_by_xpath('//td[@class="smmerchant"]')
for info in seller_info:
    try:
        Seller.append(info.text)
    except NoSuchElementException   as e:
        Seller.append('-')
    
price = driver.find_elements_by_xpath('//td[@class="smprice"]')
for prices in price:
    try:
        Price.append(prices.text)
    except NoSuchElementException   as e:
        Price.append('-') 
    
Availability_tag=driver.find_elements_by_xpath("//p[@style='margin: 0px 0 0 0;font-size: 13px;position: relative;font-weight: 400;line-height: 19px;width: 89px;']")
for i in Availability_tag:
    try:
        Availabilitytag1=i.text
        Availability.append(Availabilitytag1.replace('\n',""))
    except NoSuchElementException   as e:
        Availability.append('-')


In [54]:
#creating the dataset
Top_10_Gaming_Laptop=pd.DataFrame({})
Top_10_Gaming_Laptop['Product']=Product[:10]
Top_10_Gaming_Laptop['Processor']=Processor[:10]
Top_10_Gaming_Laptop['Memory']=Memory[:10]
Top_10_Gaming_Laptop['OS']=OS[:10]
Top_10_Gaming_Laptop['Display']=Display[:10]
Top_10_Gaming_Laptop['Seller']=Seller[:10]
Top_10_Gaming_Laptop['Price']=Price[:10]

In [55]:
Top_10_Gaming_Laptop

,Product,Processor,Memory,OS,Display,Seller,Price
0,ACER NITRO 5 RYZEN 9 (2021),AMD RYZEN 9 OCTA CORE | 2.4 GHZ,1 TB HDD/16 GBGB DDR4,WINDOWS 10,"15.6"" (1920 X 1080)",Flipkart,"₹ 129,990"
1,MSI STEALTH 15M 11TH GEN CORE I7-11375H (2021,INTEL CORE I7 11TH GEN - 11375H | NA,1 TB SSD/16 GBGB DDR4,WINDOWS 10,"15.6"" (1920 X 1080)",Flipkart,"₹ 134,990"
2,ASUS ROG STRIX SCAR 15 RYZEN 9-5900HX (2021),AMD RYZEN 9 OCTA CORE - 5900HX | 3.3 GHZ,2 TB SSD/32 GBGB DDR4,WINDOWS 10,"15.6"" (2560 X 1440)",Flipkart,"₹ 268,990"
3,ALIENWARE AREA 51M R2,10TH GEN INTEL® CORE™ I7-10700 | 2.90 GHZ,1 TB SSD/16 GBGB DDR4,WINDOWS 10 HOME,"17.3"" (1920 X 1080)",N/A,"₹ 342,989"
4,ALIENWARE M15 R3,10TH GEN INTEL® CORE™ I9-10980HK | NA,1 TB SSD/16 GBGB DDR4,WINDOWS 10 HOME,"15.6"" (3840 X 2160)",Amazon,"₹ 319,990"
5,ASUS ROG STRIX SCAR 15,AMD RYZEN™ 9 5900HX | 3.3 GHZ,1 TB SSD/16 GBGB DDR4,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",Flipkart,"₹ 215,990"
6,ASUS ROG ZEPHYRUS G14,AMD 3RD GEN RYZEN 9 | 3.3 GHZ,1 TB SSD/16 GBGB DDR4,WINDOWS 10 HOME,"14"" (1920 X 1080)",Amazon,"₹ 164,990"
7,LENOVO LEGION 5I,10TH GEN INTEL® CORE™ I5-10300H | 2.50 GHZ,1 TB SSD/16 GBGB DDR4,WINDOWS 10 PRO,"15.6"" (1920 X 1080)",Amazon,"₹ 76,988"
8,ASUS ROG ZEPHYRUS DUO 15,INTEL CORE I7 10TH GEN 10875H | NA,512 GB SSD/4 GBGB DDR4,WINDOWS 10,"15.6"" (3840 X 1100)",Amazon,"₹ 185,000"
9,ACER ASPIRE 7 GAMING,AMD RYZEN™ 5-5500U HEXA-CORE | NA,512 GB SSD/8 GBGB DDR4,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",Amazon,"₹ 58,898"


### Q.NO. 8  Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [56]:
driver = webdriver.Chrome("chromedriver.exe")
#get the web page with given url
url = "https://www.forbes.com/?sh=55783e032254"
driver.get(url)
time.sleep(3)


#lets get option button from the page
opt_btn = driver.find_element_by_xpath("//div[@class='header__left']//button").click()

#Now select Billionaires from options
blns = driver.find_elements_by_xpath("//li[@data-title='Billionaires']//a[1]")
bln = []
for i in blns[:1]:
    bln.append(i.get_attribute("href"))
     

for i in bln:
    driver.get(i)

bln_list = driver.find_element_by_xpath("//div[@class='channel-special-feature__articles-wrap']//a").click()
time.sleep(8)

#fetch required data from table
#fetching rank
rank = []
try:
    rnk = driver.find_elements_by_xpath("//div[@class='rank']")
    for i in rnk:
        rank.append(i.text)
except NoSuchElementException:
    pass
#fetching names
names = []
try:
    name = driver.find_elements_by_xpath("//div[@class='personName']")
    for i in name:
        names.append(i.text)
except NoSuchElementException:
    pass
#fetching net worth
net_worth = []
try:
    worth = driver.find_elements_by_xpath("//div[@class='netWorth']")
    for i in worth:
        net_worth.append(i.text)
except NoSuchElementException:
    pass
#fetching age
Age = []
try:
    age = driver.find_elements_by_xpath("//div[@class='age']")
    for i in age:
        Age.append(i.text)
except NoSuchElementException:
    Age.append('-')
#fetching citizenship
citizenship = []
try:
    cit = driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for i in cit:
        citizenship.append(i.text)
except NoSuchElementException:
    citizenship.append('-')
#fetching source
source = []
try:
    src = driver.find_elements_by_xpath("//div[@class='source-column']")
    for i in src:
        source.append(i.text)
except NoSuchElementException:
    source.append('-')
#fetching Industry
industry = []
try:
    ind = driver.find_elements_by_xpath("//div[@class='category']")
    for i in ind:
        industry.append(i.text)
except NoSuchElementException:
    industry.append('-')
    
#creating dataframe
data = list(zip(rank,names,net_worth,Age,citizenship,source,industry))
df = pd.DataFrame(data, columns = ["Rank", "Name", "Net worth", "Age", "Citizenship", "Source","Industry"])

In [57]:
df

,Rank,Name,Net worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
195,195.,Harry Triguboff,$11.2 B,88,Australia,real estate,Real Estate
196,197.,Leonid Fedun & family,$11.1 B,65,Russia,oil,Energy
197,197.,Eyal Ofer,$11.1 B,70,Israel,"real estate, shipping",Diversified
198,197.,Evan Spiegel,$11.1 B,30,United States,Snapchat,Technology


### Q.NO. 9 Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [70]:
driver = webdriver.Chrome("chromedriver.exe")
driver.get("https://www.youtube.com/watch?v=YfO28Ihehbk")
driver.maximize_window()
driver.execute_script("window.scrollBy(0,10000)" "")

In [71]:
for i in range(200):
    driver.execute_script("window.scrollBy(0,10000)" "")

In [74]:
Comment = driver.find_elements_by_id("content-text")
len(Comment)

40

In [75]:
Comments = []
for i in Comment:
    Comments.append(i.text)

In [76]:
comment_upvote = driver.find_elements_by_id("vote-count-middle")
len(comment_upvote)

40

In [77]:
Comment_Upvotes = []
for i in comment_upvote:
    Comment_Upvotes.append(i.text)

In [78]:
time1 = driver.find_elements_by_xpath("//*[@id='header-author']/yt-formatted-string/a")
len(time1)

40

In [79]:
Time = []
for i in time1:
    Time.append(i.text)

In [80]:
import pandas as pd
Youtube=pd.DataFrame({})
Youtube['Comments']= Comments
Youtube['Comment Upvotes']= Comment_Upvotes
Youtube['Times']= Time

In [81]:
Youtube.head()

,Comments,Comment Upvotes,Times
0,01:04 introduction to python\n05:14 Python set...,534,2 years ago
1,Telusko and Navin I have gone through your 7 h...,6,5 months ago
2,Running out of words to appreciate your effort...,20,1 year ago
3,"Whenever it comes to any topic, I always prefe...",5,1 year ago
4,Excellent teacher makes everything so easy to ...,29,1 year ago


### Q.NO. 10 Python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [3]:
import warnings
warnings.filterwarnings('ignore')
driver = webdriver.Chrome("chromedriver.exe")
driver.get("https://www.hostelworld.com/s?q=London,%20England&country=England&city=London&type=city&id=3&from=2021-07-23&to=2021-07-26&guests=2&page=1")

In [4]:
Hostel_Name1 = driver.find_elements_by_xpath("//h2[@class='title title-6']/a")
Hostel_Name = []
for i in Hostel_Name1:
    Hostel_Name.append(i.text)  

In [5]:
Distance = driver.find_elements_by_class_name('description')

In [6]:
Distance_from_City_Centre = []

In [7]:
for i in Distance:
    Distance_from_City_Centre.append(i.text)    

In [8]:
rating = driver.find_elements_by_xpath("//div[@class='score orange big']")

In [9]:
Ratings = []
for i  in rating:
    try:
        Ratings.append(i.text)
    except NoSuchElementException as E:
        Ratings.append("__")

In [10]:
Total_Reviews1 = driver.find_elements_by_xpath("//div[@class='reviews']")

In [11]:
Total_Reviews =[]
for i in Total_Reviews1:
    Total_Reviews.append(i.text)

In [12]:
Overall_reviews1 = driver.find_elements_by_xpath("//div[@class='keyword']")

In [13]:
Overall_Reviews =[]
for i in Overall_reviews1:
    Overall_Reviews.append(i.text)

In [14]:
privates_from_price1 =driver.find_elements_by_xpath("//div[@class='price title-5']")
Privates_from_price =[]
for i in privates_from_price1:
    Privates_from_price.append(i.text)

In [15]:
Privates_from_price = Privates_from_price[0::2]

In [16]:
dorms_from_price1 = driver.find_elements_by_xpath("//div[@class='price title-5']")
Dorms_from_price =[]
for i in dorms_from_price1:
    Dorms_from_price.append(i.text)

In [17]:
Dorms_from_price = Dorms_from_price[1::2]

In [18]:
facilities1 = driver.find_elements_by_xpath("//div[@class='facilities-label facilities']")
Facilities=[]
for i in facilities1:
    Facilities.append(i.text)

In [19]:
property_description1 = driver.find_elements_by_xpath("//div[@class='rating-factors prop-card-tablet rating-factors small']")
Property_Description =[]
for i in property_description1:
    Property_Description.append(i.text)

In [20]:
import pandas as pd
Hostel=pd.DataFrame({})
Hostel['Hostel Name']= Hostel_Name[0:20]
Hostel['Distance from City Centre']= Distance_from_City_Centre[0:20]
Hostel['Ratings']= Ratings[0:20]
Hostel['Total Reviews']= Total_Reviews[0:20]
Hostel['Overall Reviews']= Overall_Reviews[0:20]
Hostel['Privates from Price']= Privates_from_price[0:20]
Hostel['Dorms from Prices']= Dorms_from_price[0:20]
Hostel['Property Description']= Property_Description[0:20]
Hostel['Facilities']= Facilities[0:20]

In [21]:
Hostel.head(10)

,Hostel Name,Distance from City Centre,Ratings,Total Reviews,Overall Reviews,Privates from Price,Dorms from Prices,Property Description,Facilities
0,Mornington Camden,Hostel - 4.1km from city centre,8.3,44 Total Reviews,Superb,Rs3341,Rs3657,Perfect Location\nSuperb Staff\nFantastic Clea...,Free WiFi
1,Smart Russell Square Hostel,Hostel - 2.6km from city centre,6.6,9555 Total Reviews,Fabulous,Rs1126,Rs4561,Perfect Location\nBrilliant Staff\nAwesome Cle...,Free WiFi\nFollows Covid-19 sanitation guidance
2,Smart Camden Inn Hostel,Hostel - 4.4km from city centre,8.4,2727 Total Reviews,Fabulous,Rs1126,Rs14692,Perfect Location\nSuperb Staff\nExcellent Clea...,Free WiFi\nFollows Covid-19 sanitation guidance
3,Selina Camden,Hostel - 5.5km from city centre,9.1,13 Total Reviews,Fabulous,Rs4521.04 Rs3843,Rs2604,Perfect Location\nSuperb Staff\nFantastic Clea...,Free WiFi
4,Queen Elizabeth Chelsea,Hostel - 5.7km from city centre,7.5,3208 Total Reviews,Good,Rs1206,Rs6631,Perfect Location\nSuperb Staff\nAwesome Cleanl...,Free WiFi
5,Pickwick Hall,Hostel - 2.3km from city centre,9.0,2266 Total Reviews,Fabulous,Rs4581.32 Rs4123,Rs1378.63 Rs1241,Perfect Location\nSuperb Staff\nFantastic Clea...,Free Breakfast
6,New Cross Inn Hostel,Hostel - 6.1km from city centre,7.2,1041 Total Reviews,Superb,Rs748,Rs6329,Wonderful Location\nMarvellous Staff\nAwesome ...,Free WiFi\nFree Breakfast
7,No.8 Willesden Hostel London,Hostel - 10km from city centre,7.7,4623 Total Reviews,Very Good,Rs1808,Rs6229,Wonderful Location\nSuperb Staff\nExcellent Cl...,Free WiFi\nFollows Covid-19 sanitation guidance
8,Safestay London Elephant & Castle,Hostel - 1.7km from city centre,7.9,4014 Total Reviews,Superb,Rs2411,Rs2914,Perfect Location\nMarvellous Staff\nExcellent ...,Free WiFi\nFollows Covid-19 sanitation guidance
9,Barmy Badger Backpackers,Hostel - 5.5km from city centre,9.3,1685 Total Reviews,Very Good,Rs748,Rs7183,Perfect Location\nSuperb Staff\nFantastic Clea...,Free WiFi\nFree Breakfast
